In [1]:
import openai
import pandas as pd
import json
from IPython.display import Markdown
import sys

In [2]:
zz=openai.Model.list()

In [3]:
len(zz['data'])

62

In [4]:
[zz['data'][idx]['id'] for idx in range(len(zz['data'])) if 'gpt' in zz['data'][idx]['id']]

['gpt-4',
 'gpt-4-0613',
 'gpt-4-0314',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo']

In [5]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,
}

In [6]:
def print_save(string, file='../cmp2.md'):
    print(string)
    with open(file,'w') as f:
        f.write(string)

## Load `prompt_template.md`

In [7]:
sys.path.append('..')
from utils import *
from functools import partial

In [8]:
# prompt_dict=load_prompt_template('../prompt_template.md')
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [30]:
prompt_dict().keys()

dict_keys(['Preamble', 'Construct Kinetic Hamiltonian (continuum version)', 'Construct Kinetic Hamiltonian (lattice version)', 'Define each term in Kinetic Hamiltonian (continuum version)', 'Construct Potential Hamiltonian (continuum version)', 'Define each term in Potential Hamiltonian (continuum version)', 'Construct interaction Hamiltonian (real space, lattice version)', 'Construct interaction Hamiltonian (momentum space)', 'Convert from single-particle to second-quantized form, return in matrix', 'Convert from single-particle to second-quantized form, return in summation (expand the matrix)', 'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)', 'Convert noninteracting Hamiltonian in real space to momentum space (lattice version)', 'Convert interacting Hamiltonian in real space to momentum space (lattice version)', 'Particle-hole transformation', 'Simplify the Hamiltonian in the particle-hole basis', "Wick's theorem", 'Extract quadratic term', 'H

In [9]:
print(prompt_dict['Construct Kinetic Hamiltonian (continuum version)'])

You will be instructed to describe the kinetic term of Hamiltonian in {system} in the {real|momentum} space in the {single-particle|second-quantized} form.   
The degrees of freedom of the system are: {dof}  
Express the Kinetic Hamiltonian {kinetic_symbol} using {var} which are only on the diagonal terms, and arrange the basis in the order of {order}.

Use the following conventions for the symbols:  
{def_var}




In [11]:
df=pd.DataFrame(list(prompt_dict.items()),columns=['Task','Prompt'])

## Construct Kinetic Hamiltonian (continuum version)

In [242]:
user_msg=[]
AI_msg=[]
sys_msg=[{'role': 'system', 'content': prompt_dict()['Preamble']}]
kwargs_list=[]

In [243]:
kwargs = {
    'task':    'Construct Kinetic Hamiltonian (continuum version)',
    'system':    'the hole-doped AB-stacked MoTe2/WSe2',
    'real|momentum':    'real',
    'single-particle|second-quantized':    'single-particle',
    'dof':    'valley index (+K and -K valley), layer index (top and bottom layer)',
    'kinetic_symbol':    '$H_{Kinetic}(r)$',
    'var':    r'$E_{\tau,l}$',
    'order':    '(+K,bottom), (+K,top), (-K,bottom), (-K,top)',
    'def_var':
    r'''$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$''',
    'answer':    '$H_{Kinetic}(r) = \\begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\\\ 0 & E_{+K,t} & 0 & 0 \\\\ 0 & 0 & E_{-K,b} & 0 \\\\ 0 & 0 & 0 & E_{-K,t} \\end{pmatrix}$'
}


In [244]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [245]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [246]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [247]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [248]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [249]:
# rs['choices'][0]['message'].content

In [250]:
kwargs_list.append(kwargs)

## Define each term in Kinetic Hamiltonian (continuum version)

In [251]:
# user_msg=[]
# AI_msg=[]
# sys_msg=[{'role': 'system', 'content': prompt_dict()['Preamble']}]
# kwargs_list=[]

In [252]:
kwargs = {
    'task': 'Define each term in Kinetic Hamiltonian (continuum version)',
    'var':r' four $E_{\tau,l}(k)$',
    'Description':r'''For all energy dispersions, $E_{l,\tau}(k)$, it characterizes the dispersion for free holes, namely, $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$.   
In addition, for the top layer, $E_{\tau,t}(k)$, there is a shift of momentum $k$, to make the band bottom at $\tilde{\tau}\kappa$. You should infer the correct $\tilde{\tau}$.  
Finally, in the real space, the momentum $k=-i \partial_r$. You should keep the form of $k$ in the Hamiltonian for short notations but should remember $k$ is an operator.  ''',
    'expression_kinetic': '$H_{Kinetic}(r) = \\begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\\\ 0 & E_{+K,t} & 0 & 0 \\\\ 0 & 0 & E_{-K,b} & 0 \\\\ 0 & 0 & 0 & E_{-K,t} \\end{pmatrix}$',
    'kinetic_symbol': r'$H_{Kinetic}(r)$',
    'def_var':r'''$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone  ''',
'answer':'$H_{Kinetic}(r) = \\begin{pmatrix} -\\frac{\\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\\\ 0 & -\\frac{\\hbar^2 (k-\\kappa)^2}{2 m_t} & 0 & 0 \\\\ 0 & 0 & -\\frac{\\hbar^2 k^2}{2 m_b} & 0 \\\\ 0 & 0 & 0 & -\\frac{\\hbar^2 (k+\\kappa)^2}{2 m_t} \\end{pmatrix}$'
}

In [253]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [254]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [255]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [256]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)
# Markdown(rs['choices'][0]['message'].content)

In [257]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [258]:
# rs['choices'][0]['message'].content

In [259]:
kwargs_list.append(kwargs)

## Construct Potential Hamiltonian (continuum version)

In [260]:
# user_msg=[]
# AI_msg=[]
# sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
# kwargs_list=[]

In [261]:
kwargs = {
    'task':
    'Construct Potential Hamiltonian (continuum version)',
    'system':    'the hole-doped AB-stacked MoTe2/WSe2',
    'real|momentum':'real',
    'single-particle|second-quantized':'single-particle',
    'kinetic_symbol': r'$H_{Kinetic}(r)$',
    
    'Description': r'''At each valley, it is a 2 by 2 matrix $H_{Potential,\tau}(r)$, where each component is layer index. The diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is  intralayer tunneling $\Delta_l(r)$. The off-diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is interlayer potential $\Delta_{T,\tau}(r)$, which should be kept hermitian.  
There is no hybridization between $+K$ valley and $-K$ valley, i.e., the corresponding matrix elements in $H_{Potential}(r)$ are all zero''',
    'potential_symbol':'$H_{Potential}(r)$',
    'var':
    r'$\Delta_l(r)$ and $\Delta_{T,\tau}(r)$',
    'def_var': r'''  
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.  ''',
    'answer':'$H_{Potential}(r) = \\begin{pmatrix} \\Delta_b(r) & \\Delta_{T,+K}(r) & 0 & 0 \\\\ \\Delta_{T,+K}^*(r) & \\Delta_t(r) & 0 & 0 \\\\ 0 & 0 & \\Delta_b(r) & \\Delta_{T,-K}(r) \\\\ 0 & 0 & \\Delta_{T,-K}^*(r) & \\Delta_t(r) \\end{pmatrix}$'
}


In [262]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to describe the potential term of Hamiltonian $H_{Potential}(r)$ in the real space in the single-particle form.  
The potential Hamiltonian has the same degrees of freedom as the kinetic Hamiltonian $H_{Kinetic}(r)$.  
At each valley, it is a 2 by 2 matrix $H_{Potential,\tau}(r)$, where each component is layer index. The diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is  intralayer tunneling $\Delta_l(r)$. The off-diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is interlayer potential $\Delta_{T,\tau}(r)$, which should be kept hermitian.  
There is no hybridization between $+K$ valley and $-K$ valley, i.e., the corresponding matrix elements in $H_{Potential}(r)$ are all zero.  
Express the potential Hamiltonian $H_{Potential}(r)$ using $\Delta_l(r)$ and $\Delta_{T,\tau}(r)$.  

Use the following conventions for the symbols (You should also remember the conventions in my previous prompts if there are no conflicts. If you have conflicts in 

In [263]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [264]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [265]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [266]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [267]:
# rs['choices'][0]['message'].content

In [268]:
kwargs_list.append(kwargs)

## Define each term in Potential Hamiltonian (continuum version)

In [269]:
# user_msg=[]
# AI_msg=[]
# sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
# kwargs_list=[]

In [270]:
kwargs = {
    'task':
    'Define each term in Potential Hamiltonian (continuum version)',
    'potential_symbol':'$H_{Potential}(r)$',
    'var':    r'the intralayer potential $\Delta_l(r)$ and interlayer tunneling $\Delta_{T,\tau}(r)$',
    'real|momentum':'real',
    'single-particle|second-quantized':'single-particle',
    'kinetic_symbol':    '$H_{Kinetic}(r)$',
    'Description': r'''The intralayer on the top layer $\Delta_t(r)$ is $V_{zt}$.  
The intralayer on the bottom layer $\Delta_b(r)$ is $2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b)$.  
The interlayer tunneling at valley $\tau$ is $\tilde{\tau}w(1+\omega^{\tilde{\tau}}e^{i\tilde{\tau} g_2 \cdot r} + \omega^{2\tilde{\tau}} e^{i\tilde{\tau} g_3 \cdot r})$. You should infer the correct $\tilde{\tau}$ such that $\tilde{\tau}$ does not appear in the answer.  ''',
    'def_var': r'''$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength''',
    'expression_Potential': '$H_{Potential}(r) = \\begin{pmatrix} \\Delta_b(r) & \\Delta_{T,+K}(r) & 0 & 0 \\\\ \\Delta_{T,+K}^*(r) & \\Delta_t(r) & 0 & 0 \\\\ 0 & 0 & \\Delta_b(r) & \\Delta_{T,-K}(r) \\\\ 0 & 0 & \\Delta_{T,-K}^*(r) & \\Delta_t(r) \\end{pmatrix}$',
    'answer':'$H_{Potential}(r) = \\begin{pmatrix} 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b) & w(1+\\omega e^{i g_2 \\cdot r} + \\omega^2 e^{i g_3 \\cdot r}) & 0 & 0 \\\\ w(1+\\omega^{-1}e^{-i g_2 \\cdot r} + \\omega^{-2} e^{-i g_3 \\cdot r}) & V_{zt} & 0 & 0 \\\\ 0 & 0 & 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b) & -w(1+\\omega^{-1}e^{-i g_2 \\cdot r} + \\omega^{-2} e^{-i g_3 \\cdot r}) \\\\ 0 & 0 & -w(1+\\omega e^{i g_2 \\cdot r} + \\omega^2 e^{i g_3 \\cdot r}) & V_{zt} \\end{pmatrix}$'
}


In [271]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [272]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [273]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [274]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [275]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [276]:
# rs['choices'][0]['message'].content

In [277]:
kwargs_list.append(kwargs)

## Convert from single-particle to second-quantized form, return in matrix

In [278]:
kwargs = {
    'task':'Convert from single-particle to second-quantized form, return in matrix',
    'real|momentum':'real',
    'nonint_symbol': r'$H^{0}(r)$',
    'potential_symbol':'$H_{Potential}(r)$',
    'kinetic_symbol':'$H_{Kinetic}(r)$',
    '$r$|$k$': '$r$',
    'second_nonint_symbol': r'$\hat{H}^{0}$',
    'def_var': r'''$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form ''',
    'answer':r'$\hat{H}^{0} = \int dr \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$'
}


In [279]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [280]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [281]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [282]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [283]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [284]:
# rs['choices'][0]['message'].content

In [285]:
kwargs_list.append(kwargs)

## Convert from single-particle to second-quantized form, return in summation (expand the matrix)

In [286]:
kwargs = {
    'task':'Convert from single-particle to second-quantized form, return in summation (expand the matrix)',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'matrix_element_symbol': r'$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$',
    'basis_symbol': r'$\psi_{\tau,l}(r)$',
    'expression_second_nonint': r'$\hat{H}^{0} = \int dr \vec{\psi}^\dagger(r) H^{0}(r) \vec{\psi}(r)$, where $H^{0}(r)=H_{Kinetic}(r)+H_{Potential}(r)$, $H_{Kinetic}(r) = \begin{pmatrix} -\frac{\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\ 0 & -\frac{\hbar^2 (k-\kappa)^2}{2 m_t} & 0 & 0 \\ 0 & 0 & -\frac{\hbar^2 k^2}{2 m_b} & 0 \\ 0 & 0 & 0 & -\frac{\hbar^2 (k+\kappa)^2}{2 m_t} \end{pmatrix}$, and $H_{Potential}(r) = \begin{pmatrix} 2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b) & w(1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r}) & 0 & 0 \\ w(1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r}) & V_{zt} & 0 & 0 \\ 0 & 0 & 2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b) & -w(1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r}) \\ 0 & 0 & -w(1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r}) & V_{zt} \end{pmatrix}$',
    'def_var': r'''$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  ''',
    'answer':r'$\hat{H}^{0} = \int dr \sum_{l_1, l_2} \left( \psi_{+K,l_1}^\dagger(r) H^{0}_{+K,l_1,+K,l_2}(r) \psi_{+K,l_2}(r) + \psi_{-K,l_1}^\dagger(r) H^{0}_{-K,l_1,-K,l_2}(r) \psi_{-K,l_2}(r) \right)$'
}


In [287]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [288]:
kwargs_list.append(kwargs)

## Convert noninteracting Hamiltonian in real space to momentum space (continuum version)

In [289]:
kwargs = {
    'task':'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)',
    'real_creation_op': r'$\psi_{\tau,l}^\dagger(r)$',
    'momentum_creation_op': r'$c_{\tau,l}^\dagger(k)$',
    'def_FT': r'$c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$',
    'real_var': '$r$',
    'entire_real|fBZ': 'entire real space',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'def_var': r'''$c_{l,\tau}(k)$ : electron annihilation operator at momentum $k$, layer $l$, and valley $\tau$  
$c_{l,\tau}^\dagger(k)$ : electron creation operator at momentum $k$, layer $l$, and valley $\tau$  
$V$ : area of unit cell in the real space  ''',
    'answer':r'$\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} c_{\tau,l_1}^\dagger(k_1) H^{0}_{\tau,l_1,\tau,l_2}(k_1,k_2) c_{\tau,l_2}(k_2)$'
}


In [290]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [291]:
kwargs_list.append(kwargs)

## Particle-hole transformation

In [292]:
kwargs = {
    'task':'Particle-hole transformation',
    'hole_op': r'$b_{l,\tau}(k)$',
    'particle_op': r'$c_{l,\tau}^\dagger(k)$',
    'particle_creation_op': r'$c_{l,\tau}$',
    'hole_creation_op': r'$b_{l,\tau}(k)^\dagger$',
    'particle_annihilation_op': r'$c_{l,\tau}^\dagger$',
    'hole_annihilation_op': r'$b_{l,\tau}(k)$',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'def_var': r'''$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$''',
    'expression_particle_Ham' :r'$\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} c_{\tau,l_1}^\dagger(k_1) H^{0}_{\tau,l_1,\tau,l_2}(k_1,k_2) c_{\tau,l_2}(k_2)$',
    'answer':r'$\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}(k_1) H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2) b_{\tau, l_2}^\dagger(k_2)$'
}


In [293]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [294]:
kwargs_list.append(kwargs)

## Simplify the Hamiltonian in the particle-hole basis

In [295]:
kwargs = {
    'task':'Simplify the Hamiltonian in the particle-hole basis',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'hole_op': r'$b_{l,\tau}(k)$',
    'index_1': r'$l_1$/$k_1$',
    'index_2': r'$l_2$/$k_2$',
    'op': r'$b$',
    'Ham_op': r'$H$',
    'def_var': r'''$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise.  
$`[b_i, b_j]_{+}`$ : anticommuting relation, i.e., $`[b_i, b_j]_+= b_i b_j +b_j b_i`$  
$\hat{H}^{0,final}$ : the final version of noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification.''',
    'expression_hole_Ham': r'$\hat{H}^{0} = \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}(k_1) H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2) b_{\tau, l_2}^\dagger(k_2)$',
    'answer':r'$\hat{H}^{0} = \sum_{\tau, l_1, k_1} H^{0}_{\tau, l_1, \tau, l_1}(k_1, k_1) - \sum_{\tau, l_1, l_2, k_1, k_2} b_{\tau, l_1}^\dagger(k_1) (H^{0}_{\tau, l_1, \tau, l_2}(k_1, k_2))^* b_{\tau, l_2}(k_2)$'
}


In [296]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [297]:
kwargs_list.append(kwargs)

## Construct interaction Hamiltonian (momentum space)

In [298]:
kwargs = {
    'task':'Construct interaction Hamiltonian (momentum space)',
    'second_int_symbol':  r'$\hat{H}^{int}$',
    'index': r'valley index $\tau_i$ and layer index $l_i$',
    'momentum': r'momentum $k_i$',
    'For each operator, the total momentum is the sum of moire reciprocal lattice $b_i$ and momentum with in the first BZ $k_i$|None': '',
    'interaction': r'the dual-gate-screened Coulomb interaction',
    'int_form': r'$2\pi e^2\tanh(|q| d)/(\epsilon |q|)$',
    'normalization_factor': r'$\frac{1}{2 V}$',
    'op': r'$b_{l,\tau}(k)$',
    'def_var': r'''$d$ : distance between dual gates  
$\epsilon$ : dielectric constant  
$V(q)$ : dual-gate-screened Coulomb interaction in the momentum space''',
    'answer':r'$\hat{H}^{int} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} b_{l_1,\tau_1}^\dagger(k_1)b_{l_2,\tau_2}^\dagger(k_2)b_{l_2,\tau_2}(k_3)b_{l_1,\tau_1}(k_4) V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$'
}


In [299]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [300]:
kwargs_list.append(kwargs)

## Wick's theorem

In [301]:
kwargs = {
    'task':"Wick's theorem",
    'second_int_symbol':  r'$\hat{H}^{int}$',
    'HF_symbol': r'$\hat{H}^{int,HF}$',
    'expression_int': r'$\hat{H}^{int} = \frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} b_{l_1,\tau_1}^\dagger(k_1)b_{l_2,\tau_2}^\dagger(k_2)b_{l_2,\tau_2}(k_3)b_{l_1,\tau_1}(k_4) V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$',
    'def_var': r'''$\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_2) \rangle$ : expectation of $b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_2)$ over the many-body ground states.  
$\hat{H}^{int,HF}$ : Interaction term after Hartree-Fock approximation''',
    'answer':r'$\hat{H}^{int,HF}=\frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4)-\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) + \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle\right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$'
}


In [302]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [303]:
kwargs_list.append(kwargs)

## Extract quadratic term

In [304]:
kwargs = {
    'task':"Extract quadratic term",
    'HF_symbol': r'$\hat{H}^{int,HF}$',
    'bilinear_op': r'$b^\dagger b$',
    'HF_2_symbol': r'$\hat{H}^{int,HF,2}$',
    'expression_HF': r'$\hat{H}^{int,HF}=\frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4)-\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) + \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle\right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$',
    'def_var': r'''$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$''',
    'answer':r'$\hat{H}^{int,HF,2}=\frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3)\right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$'
}


In [305]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [306]:
kwargs_list.append(kwargs)

## Swap the index to combine Hartree and Fock terms

In [307]:
kwargs = {
    'task':"Swap the index to combine Hartree and Fock terms",
    'HF_2_symbol': r'$\hat{H}^{int,HF,2}$',
    'expval': r'$\langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4) \rangle$',
    'expression_Hartree_1':r'$\langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4) \rangle b_{\tau_2,l_2}^\dagger(k_2) b_{\tau_2,l_2}(k_3)$',
    'expression_Hartree_2':r'$\langle b_{\tau_2,l_2}^\dagger(k_2) b_{\tau_2,l_2}(k_3) \rangle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4)$',
    'expression_HF_2':r'$\hat{H}^{int,HF,2}=\frac{1}{2V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) + \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) - \langle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4) \rangle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3)\right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$',
    'answer':r'$\hat{H}^{int,HF,2}=\frac{1}{V}\sum_{\tau_1,\tau_2,l_1,l_2,k_1,k_2,k_3,k_4} \left[\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_1,\tau_1}(k_4) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_2,\tau_2}(k_3) - \langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_3) \rangle b_{l_2,\tau_2}^\dagger(k_2) b_{l_1,\tau_1}(k_4)\right] V(|k_1-k_4|)\delta_{k_1+k_2,k_3+k_4}$'
}


In [308]:
# print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

In [309]:
kwargs_list.append(kwargs)

In [310]:
len(kwargs_list)

13

In [425]:
--

SyntaxError: invalid syntax (3659366440.py, line 1)

## Save jsonl

In [311]:
with open('2111.01152.jsonl', 'w') as f:
    for dict_obj in kwargs_list:
        f.write(json.dumps(dict_obj) + '\n')

# Draft

In [ ]:
with open('1106.6060/1106.6060.jsonl', 'w') as f:
    json.dump(kwargs,f)

In [ ]:
with open('cmp2.md','w') as f:
    f.write(generate_prompt(kwargs)['content'])

In [26]:
preamble=r"""You are a physicist helping me to construct Hamiltonian and perform Hartree-Fock step by step based on my instructions. 
You should follow the instruction strictly.
Your reply should be succinct while complete. You should not expand any unwanted content.
You will be learning background knowledge by examples if necessary.
Confirm and repeat your duty if you understand it."""

In [20]:
template=r"""You will be instructed to describe the kinetic term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in the real space in the single-particle form.   
The degrees of freedom of the system are: valley index (+K and -K valley), layer index (top and bottom layer). 
Express the Kinetic Hamiltonian $H_{Kinetic}(r)$ using $E_{\tau,l}$ which are only on the diagonal terms, and arrange the basis in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top).

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{l,\tau}$ : energy dispersion for layer $l$ and valley $\tau$   """

In [21]:
rs=openai.ChatCompletion.create(model="gpt-4", temperature=0,messages=[{'role':'system','content':preamble},{'role':'user', 'content':template}])

In [22]:
Markdown(rs['choices'][0]['message'].content)

Understood. The kinetic Hamiltonian $H_{Kinetic}(r)$ in the real space for the hole-doped AB-stacked MoTe2/WSe2 system can be written in the single-particle form as follows:

$H_{Kinetic}(r) = \begin{pmatrix} E_{b,+K} & 0 & 0 & 0 \\ 0 & E_{t,+K} & 0 & 0 \\ 0 & 0 & E_{b,-K} & 0 \\ 0 & 0 & 0 & E_{t,-K} \end{pmatrix}$

Here, the basis is arranged in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top). The diagonal terms represent the energy dispersion for each combination of layer and valley.

In [10]:
template=r'''Describe the kinetic term of free fermion Hamiltonian in the AB-stacked MoTe$_2$/WSe$_2$. 
This system has a spin-valley locking, so you should construct a continuum Hamiltonian which contains two valleys.
Each valley is a 2 by 2 matrix with each component for the top or bottom layer.
The kinetic term is hole-like free fermion dispersion for both layers, with a minus sign before. 
There is a momentum shift on the top layer to make the band bottom at the corner of the moire Brillouin zone.
Return a 4 by 4 matrix, where the basis is (+K, bottom), (+K, top), (-K, bottom), (-K, top). +K and -K are two valleys in the corners of monolayer moire Brillouin zone.


Use the following conventions for the symbols:
$H_\tau$ : Valley-dependent free Hamiltonian 
$\tau$ : valley index
$m_b$ ($m_t$) : effective mass on the bottom (top) layer
$\kappa$ : corner of moire Brillouin Zone
$ r $ : real space position
$ k $ : momentum operator
'''

In [11]:
print(template)

Describe the kinetic term of free fermion Hamiltonian in the AB-stacked MoTe$_2$/WSe$_2$. 
This system has a spin-valley locking, so you should construct a continuum Hamiltonian which contains two valleys.
Each valley is a 2 by 2 matrix with each component for the top or bottom layer.
The kinetic term is hole-like free fermion dispersion for both layers, with a minus sign before. 
There is a momentum shift on the top layer to make the band bottom at the corner of the moire Brillouin zone.
Return a 4 by 4 matrix, where the basis is (+K, bottom), (+K, top), (-K, bottom), (-K, top). +K and -K are two valleys in the corners of monolayer moire Brillouin zone.


Use the following conventions for the symbols:
$H_\tau$ : Valley-dependent free Hamiltonian 
$\tau$ : valley index
$m_b$ ($m_t$) : effective mass on the bottom (top) layer
$\kappa$ : corner of moire Brillouin Zone
$ r $ : real space position
$ k $ : momentum operator



In [48]:
x=0
print(f' {x} {{}} {{{x}}}')

 0 {} {0}
